# Batch run of guide RNA design based on dbSNP rs numbers  

In [1]:
from IPython.core.display import HTML
HTML('''
<style>
    div.prompt {display:none}
    div.cell{
        width:100%;
        margin-left:1%;
        margin-right:1%;
    }
</style>''')

# shift code cells
HTML('''
<style>
    div.input{
        width:100%;
        padding-left:2em;
        padding-right:0em;
    }
</style>''')
# code toggle button
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').show();
 } else {
 $('div.input').hide();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="See usage information at the end! Click here to toggle on/off the raw code. "></form>''')

In [2]:
from Bio import Seq
import re
from itertools import product
import pandas as pd
#  pip3.4  install sqlalchemy --user
# pip3.4  install pymysql --user 
from sqlalchemy import create_engine
import requests
import xml.etree.ElementTree as ET
from ipywidgets import *  
from IPython.display import display 

In [3]:
# Get genomic position from UCSC based on dbSNP rs number
# UCSC dbSNP table schema: http://ucscbrowser.genap.ca/cgi-bin/hgTables?db=hg38&hgta_group=varRep&hgta_track=snp142Common&hgta_table=snp142Common&hgta_doSchema=describe+table+schema
# dbSnpVersion=hg38.snp142 or similar
def getDbSnpPosition(rsIdList,dbSnpVersion):
    rsIdStr = ""
    for id in rsIdList:
        rsIdStr += "'" + id + "',"
    rsIdStr = rsIdStr[:-1]
    #print(rsIdStr)
    
    engine = create_engine("mysql+pymysql://genome@genome-mysql.cse.ucsc.edu")
    query = "select chrom, chromStart, chromEnd, name, observed from " + dbSnpVersion + " WHERE name IN (" + rsIdStr + ")"
    dbSNP_data =  pd.read_sql_query(query, engine)
    #chromRS.value = dbSNP_data.ix[0,0] # dbSNP_data['chrom']
    #chromRSStart.value = dbSNP_data.ix[0,1] #dbSNP_data['chromStart']
    #chromRSEnd.value = dbSNP_data.ix[0,2] #dbSNP_data['chromEnd']
    return dbSNP_data


In [4]:
rsList = ['rs339331','rs1109815','rs877343','rs2474694','rs9967549','rs2427530','rs2427531','rs78416326','rs339297','rs111770284','rs1442744','rs551721127','rs10123735','rs11168962','rs2474694','rs11672691','rs887391','rs4845688','rs1109815','rs877343','rs4845695','rs705094','rs75711918','rs1534642','rs7591175','rs6719046','rs386648104','rs386648104','rs6733913']

In [5]:
motifText = 'SpCas9,GNNNNNNNNNNNNNNNNNNNNGG,1,20'
motifText = motifText + '\n' + 'Sa23Cas9,GNNNNNNNNNNNNNNNNNNNNNNNNGRRT,1,23'
motifText = motifText + '\n' + 'Sa22Cas9,GNNNNNNNNNNNNNNNNNNNNNNNGRRT,1,22'
motifText = motifText + '\n' + 'Sa21Cas9,GNNNNNNNNNNNNNNNNNNNNNNGRRT,1,21'
motifText = motifText + '\n' + 'AsCpf1,TTTNNNNNNNNNNNNNNNNNNNN,5,19'

In [6]:
# funcion to extend ambiguous IUPAC letters. This function generates all possible sequences, 
# expanding the characters defined by the IUPAC standard, expect 'N' (since that would create a huge overhead)
def extend_ambiguous_expectN_dna(seq):
    """return list of all possible sequences given an ambiguous DNA input"""
    d = Seq.IUPAC.IUPACData.ambiguous_dna_values
    d['N'] = 'N' # here we exclude expansion of N
    r = []
    for i in product(*[d[j] for j in seq]):
        r.append("".join(i))
    return r 

In [22]:
# handle Start button click. Get rs positions, get sequences, run the motif match and updates the Results
def on_button_clicked(b):
    rsList1 = rsBox.value.split('\n')
    print(rsBox.value.split('\n'))
    rsPosDf = getDbSnpPosition(rsList1,str(dbSNPname.value))
    res = ''
    res2 = ''
    res3 = ''
    #res = res + 'browser position '+str(chrom.value)+':'+str(chrPos.value) +'-' + str(chrPos.value+len(cleanSequence))+'\n'
    res = res + 'browser hide all\n'
    res = res + 'track name="guideRNA" description="Color by strand" '+ 'visibility=2 colorByStrand="255,0,0 0,0,255"\n'
    # iterate on SNP-s. Chrom,chromStart,chromPos,rsID are given. Get the sequence between [chromStart-window, chromStart+window]
    for rs in rsPosDf.iterrows():
        chrom = rs[1][0] 
        offs = int(seqWindow.value)
        chromStartInt = int(rs[1][1]) - offs
        chromStart = str(chromStartInt)
        chromEndInt = int(rs[1][1]) + offs
        chromEnd = str(chromEndInt)
        rsIdName = rs[1][3] + '_'+ rs[1][4]
        rsStart = rs[1][1]
        rsEnd = rs[1][2]
        genomeVersion = str(dbSNPname.value).split('.')[0]
        url = 'http://genome.ucsc.edu/cgi-bin/das/' + genomeVersion \
            + '/dna?segment=' + chrom + ':' + chromStart + ',' \
            + chromEnd
        response = requests.get(url)
        SeqXml = ET.fromstring(response.text)
        SeqText = SeqXml.find('SEQUENCE').find('DNA').text
        cleanSequence = re.sub(r"[^ACTGRYSWKMBDHVN]", '',SeqText.upper())
        #print(rs[1][3],'\n',sequence)
        seqRef = cleanSequence[:]
        #seqMut = cleanSequence[:]
        if chromEndInt > rsEnd:
            # to be sure, that the mutated part does not go longer than the end of the sequence
            # otherwise we keep the original sequences
            variants = rs[1][4].split('/')
            variantSeqs = []
            for varis in variants:
                variantSeqs.append(seqRef[:offs+1] + varis.lower() + seqRef[offs+1+len(varis):])
            #ref = variants[0].lower()
            #mut = variants[1].lower()
            #seqRef = seqRef[:offs+1] + ref + seqRef[offs+1+len(ref):]
            #seqMut = seqMut[:offs+1] + mut + seqMut[offs+1+len(mut):]
        
        res = res +  chrom + '\t' + str(rsStart) + '\t' + \
                        str(rsEnd) + '\t' + rsIdName + '\t0\t+\n'

        rsMotifMatchCount = 0
        # we have the sequence, iterate through motifs
        for motifLine in motifBox.value.split('\n'):
            motifDesc = motifLine.split(',')
            motifName = motifDesc[0]
            motif = motifDesc[1]
            motifStart = int(motifDesc[2])
            motifLen = int(motifDesc[3])
            cleanMotif = re.sub(r"[^ACTGRYSWKMBDHVN]", '',motif.upper())
            cleanMotif_rc = str(Seq.Seq(cleanMotif,Seq.IUPAC.ambiguous_dna).reverse_complement())

            # forward
            i = 1
            for iupacMotif in extend_ambiguous_expectN_dna(cleanMotif):
                regexMotif = re.sub(r"N", '.',iupacMotif)
                #for match in re.finditer(regexMotif, cleanSequence): # finds only non-overlapping matches
                for match in re.finditer(r'(?=('+regexMotif+'))', cleanSequence):
                    mm = match.group(1)
                    # we need only certain part of the matched sequence, defined by outStart and outLength (0 based indexing)
                    idx1 = motifStart - 1
                    idx2 = idx1 + motifLen
                    res = res +  chrom + '\t' + str(chromStartInt+match.start()+idx1) + '\t' + \
                        str(chromStartInt+match.start()+idx2-1) + '\t' + rsIdName + '_' + motifName + '_' + 'gRNA_' + \
                        str(i) + '\t0\t+\n'
                    #?seqNoRC = str(mm[idx1:idx2])
                    
                    seqRef = variantSeqs[0]
                    seqMut1 = variantSeqs[1]
                    refMotif = seqRef[match.start()+idx1:match.start()+idx2]
                    mutMotif1 = seqMut1[match.start()+idx1:match.start()+idx2]
                    ref = variants[0].upper()
                    res2 = res2 + rsIdName + '\t' + motifName + '_' + 'gRNA_' + ref +'_'+ str(i) + '\t' + refMotif + '\n'
                    if refMotif != mutMotif1:
                        vv = 1
                        for seqMut in variantSeqs[1:]:
                            mutMotif = seqMut[match.start()+idx1:match.start()+idx2]
                            mut = variants[vv].upper()
                            vv = vv + 1
                            res2 = res2 + rsIdName \
                            + '\t' + motifName + '_' + 'gRNA_' + mut +'_'+ str(i) + '\t' + mutMotif + '\n'
                    # ' ' + cleanSequence + ' ' + seqRef + ' ' + seqMut +
                    # '_' + str(mm[idx1:idx2]) +
                    #res = res + str(match.start()+idx1+1) +' ' + str(mm[idx1:idx2]) + '+\n'
                    i = i+1
                    rsMotifMatchCount = rsMotifMatchCount + 1

            # reverse complement
            i = 1
            for iupacMotif_rc in extend_ambiguous_expectN_dna(cleanMotif_rc):
                regexMotif_rc =  re.sub(r"N", '.',iupacMotif_rc)
                #for match in re.finditer(regexMotif_rc, cleanSequence): # finds only non-overlapping matches
                for match in re.finditer(r'(?=('+regexMotif_rc+'))', cleanSequence):
                    mm = match.group(1)
                    # we need only certain part of the matched sequence, defined by motifStart and motifLen (0 based indexing)
                    idx2 = len(regexMotif_rc) - (motifStart - 1)
                    idx1 = len(regexMotif_rc) - (motifStart - 1 + motifLen)
                    res = res +  chrom + '\t' + str(chromStartInt+match.start()+idx1) + '\t' + \
                        str(chromStartInt+match.start()+idx2-1) + '\t' + rsIdName + '_' + motifName + '_' + 'gRNArc_' + \
                        str(i) + '\t0\t-\n'
                    # generate the reverse complement of the sequence
                    #?seqNoRC = str(mm[idx1:idx2])
                    #?seqRC = str(Seq.Seq(str(mm[idx1:idx2]),Seq.IUPAC.ambiguous_dna).reverse_complement())
                    
                    seqRef = variantSeqs[0]
                    seqMut1 = variantSeqs[1]
                    refMotif = seqRef[match.start()+idx1:match.start()+idx2]
                    refMotifRC = str(Seq.Seq(refMotif,Seq.IUPAC.ambiguous_dna).reverse_complement())
                    mutMotif1 = seqMut1[match.start()+idx1:match.start()+idx2]
                    mutMotif1RC = str(Seq.Seq(mutMotif1,Seq.IUPAC.ambiguous_dna).reverse_complement())
                    ref = variants[0].upper()
                    res2 = res2 + rsIdName + '\t' + motifName + '_' + 'gRNArc_' + ref +'_'+ str(i) + '\t' + refMotifRC + '\n'
                    if refMotifRC != mutMotif1RC:
                        vv = 1
                        for seqMut in variantSeqs[1:]:
                            mutMotif = seqMut[match.start()+idx1:match.start()+idx2]
                            mutMotifRC = str(Seq.Seq(mutMotif,Seq.IUPAC.ambiguous_dna).reverse_complement())
                            mut = variants[vv].upper()
                            vv = vv + 1
                            res2 = res2 + rsIdName \
                            + '\t' + motifName + '_' + 'gRNArc_' + mut +'_'+ str(i) + '\t' + mutMotifRC + '\n'
                    
                    #refMotif = seqRef[match.start()+idx1:match.start()+idx2]
                    #refMotifRC = str(Seq.Seq(refMotif,Seq.IUPAC.ambiguous_dna).reverse_complement())
                    #mutMotif = seqMut[match.start()+idx1:match.start()+idx2]
                    #mutMotifRC = str(Seq.Seq(mutMotif,Seq.IUPAC.ambiguous_dna).reverse_complement())
                    #res2 = res2 + rsIdName + '\t' + motifName + '_' + 'gRNArc_' + ref.upper() +'_'+ str(i) + '\t' + refMotifRC + '\n'
                    #if refMotifRC != mutMotifRC:
                    #    res2 = res2 + rsIdName + '\t' + motifName + '_' + 'gRNArc_' + mut.upper() +'_'+ str(i) + '\t' + mutMotifRC + '\n'
                    # ' ' + cleanSequence + ' ' + seqRef + ' ' + seqMut +
                    # '_' + str(mm[idx1:idx2]) + 
                    #res = res + str(match.start()+idx1+1) +' ' + str(mm[idx1:idx2]) + '-\n'
                    i = i+1
                    rsMotifMatchCount = rsMotifMatchCount + 1
        res3 = res3 + rsIdName + '\t' + str(rsMotifMatchCount) + '\n'
    
    resultBox.value = res
    resultBox.update_config
    tableBox.value = res2
    tableBox.update_config
    countBox.value = res3
    countBox.update_config

In [23]:
# ------------
rsIdText = ""
for id in rsList:
    rsIdText += id + '\n'
rsIdText = rsIdText[:-1]
dbSNPname = widgets.Text(description='dbSNP version:', value='hg38.snp142',width='100px', margin=8)
rsBox = widgets.Textarea(description='', value=rsIdText, width=800,height=400)
pageRS = widgets.Box(children=[widgets.HBox(children=[dbSNPname]),widgets.HBox(children=[rsBox])])
accord1 = widgets.Accordion(children=[pageRS])
accord1.set_title(0, 'dbSNP rs numbers')
# ------------

# ------------
#seqWindow = widgets.BoundedIntText(description='+/- window :', min=1, max=999999, value=25,width='100px', margin=8)
seqWindow = widgets.Text(description='+/- window around SNP:', value='25',width='100px', margin=8)
motifBox = widgets.Textarea(description='', value=motifText, width=800,height=200)
pageMotif = widgets.Box(children=[widgets.HBox(children=[seqWindow]),widgets.HBox(children=[motifBox])])
accord2 = widgets.Accordion(children=[pageMotif])
accord2.set_title(0, 'MotifID,motif,start,length')
# ------------

# ------------
startButton = widgets.Button(description='Start', 
                             width='100px',height='40px', border_width=10, border_radius = 15, margin=10)
startButton.on_click(on_button_clicked) 
startButton.background_color = "cornflowerblue"
startButton.background_color = "cornflowerblue"
startButton.button_style = 'Info'
startButton.font_weight = 'bold'
startButton.font_size = '14pt'
# ------------

resultBox = widgets.Textarea(description='', width=800,height=400)
accord3 = widgets.Accordion(children=[resultBox], width=400)
accord3.set_title(0, 'BED output')

tableBox = widgets.Textarea(description='', width=800,height=400)
accord4 = widgets.Accordion(children=[tableBox], width=400)
accord4.set_title(0, 'Tabular output')

countBox = widgets.Textarea(description='', width=800,height=400)
accord5 = widgets.Accordion(children=[countBox], width=400)
accord5.set_title(0, 'Counts')

display(accord1,accord2,startButton,accord3,accord4,accord5)

['rs339331', 'rs1109815', 'rs877343', 'rs2474694', 'rs9967549', 'rs2427530', 'rs2427531', 'rs78416326', 'rs339297', 'rs111770284', 'rs1442744', 'rs551721127', 'rs10123735', 'rs11168962', 'rs2474694', 'rs11672691', 'rs887391', 'rs4845688', 'rs1109815', 'rs877343', 'rs4845695', 'rs705094', 'rs75711918', 'rs1534642', 'rs7591175', 'rs6719046', 'rs386648104', 'rs386648104', 'rs6733913']


This tool takes a genetic _Sequence_ and looks for patterns which match the given _Motif_. In the motif 'N' character matches any letter in the sequence. User can truncate the displayed matching sequences by setting the _From_ and _Length_ fields.   
After pressing the _Start_ button, on the output the truncated matcing sequences, and their positions are listed.  

- IUPAC characters, other than ACGT are handled in Motif, but not in Sequence
- all lower case characters in the sequence are converted to upper case
- all characters except 'A','C','G','T' (including whitespaces and linebreake) will be removed from Sequence
- all characters except 'A','C','G','T' and IUPAC characters: RYSWKMBDHV, and 'N' - which matches any base will be removed from Motif
- !!! output BED files use 0-based indexing!!! See: https://genome.ucsc.edu/FAQ/FAQformat.html#format1
- in the 'Tabular' output, - strand sequences are reverse complemented
- additionally, there are 'reference' and 'mutant' sequences. The mutant position(s) are in lower  case (both for ref and mut)
- if the mutant section is outside of the cutout, than the original sequence is written out, to keep the column structure (no lowercase letters)


In [ ]:
rsIdStr = ""
for id in rsList:
    rsIdStr += "'" + id + "',"
rsIdStr = rsIdStr[:-1]
#print(rsIdStr)

engine = create_engine("mysql+pymysql://genome@genome-mysql.cse.ucsc.edu")
query = "select * from hg19.snp142 WHERE name IN (" + rsIdStr + ")"
pd.read_sql_query(query, engine)